# Basic usage of asyncio in load test scenario

You can change the following to see how the scenario behaves differently
- `num_players`
- `latency_in_secs`

All of these happens in just **1 thread** with many tasks.

Whenever you see `await`, the coroutine will `pause` at that point with it's logic being scheduled to run in a queue. At that point, it will relinquish control to the event loop. The event loop will then process other pending tasks.
Once the logic inside the coroutine is completed, the coroutine will notify the event loop that it is ready to `resume`.

The **order** of operations is guranteed for example in `player_actions`.

In [ ]:
import asyncio
import time
import random
import time

num_players = 10
latency_in_secs = 3

async def run_scenario():
    start = time.time()
    
    print("Begin background task")
    bg_future = asyncio.ensure_future(bg_task())
    
    # Players
    print("Starting {} players".format(num_players))
    player_tasks = []
    for i in range(num_players):
        player_tasks.append(asyncio.ensure_future(player_actions(i)))

    await asyncio.gather(*player_tasks)
    
    print("Cancelling background task")
    bg_future.cancel()
    
    end = time.time()

    print('Scenario took {} seconds to complete'.format(end - start))

async def player_actions(profile_id):
    await request_to_sandbox(profile_id, 'logging in')
    await request_to_sandbox(profile_id, 'searching')
    await request_to_sandbox(profile_id, 'playing a match')
    await request_to_sandbox(profile_id, 'leaving a match')

async def request_to_sandbox(profile_id, method):
    latency = random.random() * latency_in_secs
    await asyncio.sleep(latency)
    print("Player {} took {:.2f} seconds when {}".format(profile_id, latency, method))

async def bg_task():
    while(True):
        await asyncio.sleep(1)
        print("Tick")

# Main loop
loop = asyncio.get_event_loop()
loop.run_until_complete(run_scenario())